# 6-GRPO
梯度正则化偏好优化（Gradient-Regularized Preference Optimization，GRPO）是后训练阶段中，在直接偏好优化（DPO）基础上引入梯度正则化机制，进一步提升大模型偏好对齐稳定性与训练效率的策略，为解决 DPO 训练过程中易出现的梯度爆炸、过拟合等问题提供了优化方案。通过这一阶段的训练，大模型不仅能学会依照人类喜好生成回复，还能在训练过程中保持更稳定的参数更新，降低极端样本对模型偏好对齐效果的负面影响。

在这个笔记本中，我们仅对 GRPO 的训练流程进行展示和学习，因此只给出必要的代码片段，如 wandb 和 ddp 不会在此笔记本中涉及。

此笔记本的完整实现见主仓库 `/minimind/train_grpo.py`



In [1]:
# 导入依赖
import os
import platform
import argparse
import time
import math
import warnings
import re
import pandas as pd
import torch
import torch.nn.functional as F
import torch.distributed as dist
from contextlib import nullcontext

from torch import optim, nn
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data import DataLoader, DistributedSampler
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
from model.model import MiniMindLM
from model.LMConfig import LMConfig
from model.dataset import RLAIFDataset

In [2]:
warnings.filterwarnings('ignore')

## 可选参数设置

首先，查看训练的可选参数，这些参数在实际使用时通过命令行导入，为了保持笔记本的易用性，选择用 class 进行包装.

In [3]:
class args:
    epochs: int = 5 # 训练轮数，延续 sft 基础上微调
    batch_size: int = 2 # pretrain 数据集仅两个样本，设置 batch 为 2
    learning_rate: float = 8e-8 # 学习率
    device: str = 'cuda' if torch.cuda.is_available() else 'cpu'
    dtype: str = 'bfloat16' # 16 bit 浮点数：8 bit 指数 + 7 bit 尾数
    # use_wandb: bool = False # 是否使用 wandb 我们不使用
    wandb_project: str = 'MiniMind-Notebook'
    num_workers: int = 1 # 工作进程数
    # ddp：bool = False # 单机多卡
    accumulation_steps: int = 1 # 梯度累积步数
    grad_clip: float = 1.0 # 梯度剪裁
    warmup_iters: int = 0 # 学习率热启动
    log_interval: int = 1 # 每一步打印日志 仅用于观察
    local_rank: int = 1 # device 设备号
    dim: int = 512 # 词嵌入维度 模型超参数
    num_generations: int = 8 # 生成数量
    reasoning: int = 1 # 表示进行使用推理模型，若为 0 则不使用推理模型
    beta: float = 0.02 # KL惩罚系数
    n_layers: int = 1 # MiniMind Block 数量 模型超参数 | 由于 dpo 要加载两个模型 我们出于演示目的设定 n_layers = 1
    max_seq_len: int = 128 # Prompt最大长度
    max_gen_len: int = 512 # 生成文本最大长度
    use_moe: bool = False # 是否启用混合专家
    data_path: str = './toydata/rlaif_data.jsonl' # 数据集路径
    reward_model_path: str = './internlm2-1_8b-reward' # reward 模型路径
    save_dir: str = "./output"  # 模型保存目录
    save_weight: str = "minimind_rlaif"  # checkpoint 文件前缀
    save_interval: int = 1  # 每多少步保存一次模型，0表示不保存 我们这里只展示训练过程（可选择的保存模型，建议先保存）

In [4]:
print(f'查看工作设备 {args.device}')

查看工作设备 cuda


## 初始化训练

接下来，我们对一些重要模块进行初始化，我们已经了解过，分词器，模型和数据集是大模型的基本组件，我们对其进行初始化.

> 在这一阶段 我们调整的是大模型的问答偏好 因此与 sft 阶段同理 我们需要载入在 sft 阶段微调好的问答模型

In [5]:
def init_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('./model/minimind_tokenizer')
    # 初始化Policy模型
    model = MiniMindLM(lm_config)
    moe_path = '_moe' if lm_config.use_moe else ''
    ckp = f'./output/minimind_sft_{lm_config.dim}{moe_path}.pth' # 指示上一阶段训练保存的模型文件位置
    state_dict = torch.load(ckp, map_location=args.device) # 载入模型状态字典
    model.load_state_dict(state_dict, strict=False) # 装入模型
    # 初始化Reference模型
    ref_model = MiniMindLM(lm_config)
    ref_model.load_state_dict(state_dict, strict=False)
    ref_model.eval()
    ref_model.requires_grad_(False)

    print(f'LLM总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    model = model.to(args.device)
    ref_model = ref_model.to(args.device)

    return model, ref_model, tokenizer

### 奖励模型准备（必需）

已知RLAIF训练需要“奖励模型 (Reward Model)”对生成的回答进行打分。

此处选取小型且高质量的InternLM2-1.8B-Reward 
([ModelScope](https://modelscope.cn/models/Shanghai_AI_Laboratory/internlm2-1_8b-reward) | [HuggingFace](https://huggingface.co/internlm/internlm2-1_8b-reward)) 
作为基础奖励模型。

下载奖励模型后需要放置在minimind项目的**同级目录**下，推荐结构如下：

```
toy_minimind/
├── model/                      # toy_minimind 模型
│   ├── dataset.py
│   └── ...
└── internlm2-1_8b-reward/      # 奖励模型
    ├── config.json
    ├── model.safetensors
    └── ...
```

---

#### 具体操作步骤
1. 打开命令行终端（Windows 系统可按下 `Win + R`，输入 `cmd` 回车打开 CMD；macOS/Linux 打开 Terminal）。
2. 初始化 Git LFS：
   ```bash
   git lfs install
   ```
   执行成功会提示 `Git LFS initialized.`。
3. 克隆模型仓库到本地（仓库地址为 ModelScope 上的 InternLM2-1.8B-Reward 模型）：
   ```bash
   git clone https://www.modelscope.cn/Shanghai_AI_Laboratory/internlm2-1_8b-reward.git
   ```
4. 等待克隆完成：由于模型文件较大，克隆过程可能需要一定时间，期间请保持网络稳定，若出现中断可重新执行 `git clone` 命令重试。

---

In [6]:
lm_config = LMConfig(dim=args.dim, n_layers=args.n_layers, max_seq_len=args.max_seq_len + args.max_gen_len, use_moe=args.use_moe)
model, ref_model ,tokenizer = init_model(lm_config)

# Reward模型
reward_model = AutoModel.from_pretrained(
    args.reward_model_path, torch_dtype=torch.float16, trust_remote_code=True 
)
reward_model = reward_model.to(args.device).eval().requires_grad_(False)
reward_tokenizer = AutoTokenizer.from_pretrained(args.reward_model_path, trust_remote_code=True)

# 构建数据集和数据加载器
train_ds = RLAIFDataset(args.data_path, tokenizer, max_length=lm_config.max_seq_len)

train_loader = DataLoader(
    train_ds,
    batch_size=args.batch_size,
    pin_memory=True,  # 锁页内存，加速数据传输
    drop_last=False,  # 是否丢弃最后一个不完整的批次
    shuffle=False,  # RLAIF 训练不需要打乱数据 
    num_workers=args.num_workers,  # 工作进程数，0表示在主进程中加载数据
)

print(f'模型位于设备：{model.device}, 词表长度：{tokenizer.vocab_size}, DataLoader：{train_loader}')

`torch_dtype` is deprecated! Use `dtype` instead!


LLM总参数量：6.096 百万


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

模型位于设备：cuda:0, 词表长度：6400, DataLoader：<torch.utils.data.dataloader.DataLoader object at 0x000001D7CD4D7150>


In [7]:
loader = iter(train_loader)
print(f'打印一个 iter 的数据:\n{next(loader)}\n')
print(f'数据集大小：{len(train_ds)}, DataLoader 大小：{len(loader)}')

打印一个 iter 的数据:
{'prompt': ['<s>system\n你是 MiniMind，是一个有用的人工智能助手。</s>\n<s>user\n列出五个基本的人格理论，并分别以一句话概括。</s>\n<s>assistant\n', '<s>system\n你是 MiniMind，是一个有用的人工智能助手。</s>\n<s>user\n仔细阅读以下句子并回答“汤姆是医生还是建筑工人?”</s>\n<s>assistant\n'], 'answer': ['空', '空']}

数据集大小：2, DataLoader 大小：1


我们发现，train loader 的每一个 iter 都包含一个键为 `prompt` 和 `answer` 的字典，这是因为 train_dataset 每一次取数据都会返回这个包含两个列表的字典，其中：

- 列表 prompt: 每个元素是包含 system 提示、user 提问和 assistant 开头标识的完整输入文本，以特定的标签（如 `<s>` `</s>`）分隔不同角色的内容
- 列表 answer: 每个元素对应 prompt 的预期回答内容，此处两条数据的回答均为“空”

由于我们的数据集只有两条数据，而 batch size 设置为 2，因此我们的 dataloader 只有一个 iter。

# 启动训练

训练一个深度学习模型，还涉及到了优化器，损失函数和学习率调度. 接下来，我们查看 MiniMind 训练部分的代码，并进行一轮简单的训练.


In [8]:
# 学习率调度方面 采用余弦退火学习率
def get_lr(current_step, total_steps, lr):
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

# 优化器方面 选择 AdamW 优化器 并在混精度场景下创建 scaler 进行梯度缩放避免数值下溢
scaler = torch.amp.GradScaler('cuda', enabled=(args.dtype in ['float16', 'bfloat16']))  # 专门解决混合精度训练中的数值下溢问题
optimizer = optim.AdamW(model.parameters(), lr=args.learning_rate)  # AdamW 优化器

device_type = "cuda" if "cuda" in args.device else "cpu"
print(f'设备类型：{device_type}')
# 根据指定的数据类型设置混精度训练的 dtype，以下步骤为不可缺少的混精度训练准备工作
if args.dtype == 'bfloat16':
    amp_dtype = torch.bfloat16
elif args.dtype == 'float16':
    amp_dtype = torch.float16
else:
    amp_dtype = torch.float32  # 默认为 FP32
print(f'使用混精度训练，数据类型：{amp_dtype}')
# 在 cuda 上启动混精度训练，否则空白上下文
autocast_ctx = nullcontext() if device_type == "cpu" else torch.amp.autocast(device_type='cuda', dtype=amp_dtype) 

设备类型：cuda
使用混精度训练，数据类型：torch.bfloat16


GRPO（Group Relative Policy Optimization，群组相对策略优化）是DeepSeek团队提出的PPO变体，核心是用同一prompt下多输出的组内相对奖励替代绝对价值估计，无需独立价值网络，提升LLM偏好对齐的效率与稳定性。以下是其原理的结构化解析：

### 核心原理与流程
1.  **组内采样**：对每个prompt，从当前策略采样G个输出（G为超参数，常见64）构成样本组，确保组内样本同源可比。
2.  **相对奖励计算**：用奖励模型（RM）为组内输出打分，再做组内归一化（减均值、除以标准差），得到相对奖励作为优势信号，避免绝对分数波动影响。公式为：$A_i=\frac{r_i - \mu}{\sigma+\epsilon}$，其中$\mu$为组内均值，$\sigma$为组内标准差，$\epsilon$防止除零。
3. **策略优化**：沿用 PPO 的裁剪（clip）机制并加入 KL 正则。在 **token 级别**计算重要性采样比率（当前策略与**采样时旧策略**的比率），用优势加权并裁剪以限制更新幅度；同时加入与**参考策略**的 KL 散度惩罚保持稳定。目标函数为：

$$\mathcal{J}_{\text{GRPO}}(\theta)=\mathbb{E}\left[\frac{1}{G}\sum_{i=1}^{G}\frac{1}{|o_i|}\sum_{t=1}^{|o_i|}\left(\min\left(\rho_{i,t}\,\hat{A}_i,\;\text{clip}(\rho_{i,t},\,1\!-\!\varepsilon,\,1\!+\!\varepsilon)\,\hat{A}_i\right)-\beta\,D_{KL}\right)\right]$$

其中 $\rho_{i,t}=\frac{\pi_\theta(o_{i,t}|q,\,o_{i,<t})}{\pi_{\theta_{old}}(o_{i,t}|q,\,o_{i,<t})}$ 为重要性采样比率，$D_{KL}$ 为 $\pi_\theta$ 与参考策略 $\pi_{ref}$ 之间的 token 级 KL 散度，$\beta$ 为 KL 权重，$\varepsilon$ 为裁剪系数。  
4.  **迭代更新**：重复采样、计算相对奖励、优化策略，逐步提升高相对奖励输出的概率，抑制劣质输出。


---

在 GRPO 中，使用的是一种特殊的 token 级别 KL 估计器：

$$D_{KL}^{(t)} = \frac{\pi_{ref}(o_{i,t} | q, o_{i,<t})}{\pi_\theta(o_{i,t} | q, o_{i,<t})} - \log\frac{\pi_{ref}(o_{i,t} | q, o_{i,<t})}{\pi_\theta(o_{i,t} | q, o_{i,<t})} - 1$$

其中$\pi_{ref}$表示参考策略，$\pi_\theta$表示当前策略.

> 这是一种无偏的 KL 估计器，防止模型在多次训练后偏离太远，保持基本能力.


对于回答 $o_i$ 中的每个 token 位置 $t$，三个核心量的粒度各不相同：

- **$\rho_{i,t}$（重要性采样比率）**：逐 token 变化。因为在序列的不同位置上，当前策略 $\pi_\theta$ 与旧策略 $\pi_{\theta_{old}}$ 对该 token 的生成概率各自发生了不同程度的偏移，所以每个位置的比率均不相同。

- **$\hat{A}_i$（优势）**：整条回答共享。因为奖励模型（或规则）是对回答 $o_i$ 整体打分，而非对其中某个 token 单独评分，所以由此归一化得到的优势是一个序列级别的标量，回答内所有 token 共用同一个值。

- **$D_{KL}^{(t)}$（KL 散度）**：逐 token 变化。因为在序列的不同位置上，当前策略 $\pi_\theta$ 相对于参考策略 $\pi_{ref}$ 的偏离程度各不相同，每个 token 位置都有独立的散度值。

> 简而言之：$\rho_{i,t}$ 和 $D_{KL}^{(t)}$ 捕捉的是 **局部（token 级）** 信号，$\hat{A}_i$ 提供的是 **全局（序列级）** 信号；三者相乘，实现了"用全局反馈指导局部调整"。

三者的协作逻辑是：**$\hat{A}_i$ 提供全局方向（这条回答整体该被鼓励还是抑制），$\rho_{i,t}$ 和 $D_{KL}^{(t)}$ 在每个 token 位置上进行精细的局部调控**——既根据新旧策略的差异进行修正（$\rho$），又防止在某些位置上偏离参考模型过远（$D_{KL}^{(t)}$）。
- $\rho$ + clip → 控制"短期"变化（每一步不能太大）
- $D_{KL}^{(t)}$ → 控制"长期"偏移（整体不能偏太远）
> 两者结合，确保训练既能学到新东西，又不会失控。



---

In [9]:
def calculate_rewards(prompts, responses, reward_model, reward_tokenizer):
    """整合所有奖励函数计算总奖励"""
    def reasoning_model_reward(rewards):
        # 定义正则表达式模式，检查生成的文本是否包含正确的 <think> 和 <answer> 标签结构
        pattern = r"^<think>\n.*?\n</think>\n<answer>\n.*?\n</answer>$"  # .*? 表示非贪婪地匹配任意字符（除换行符外）零次或多次
        pattern2 = r"^<think>\n.*?\n</think>\n\n<answer>\n.*?\n</answer>$"
        # 检查每个生成的文本是否匹配上述模式，返回一个布尔列表
        # re.S 标志使得 . 可以匹配换行符\n，从而允许跨行匹配整个文本
        # 例如，如果 response 是 "<think>\n这是思考过程\n</think>\n<answer>\n这是答案\n</answer>"
        # 则 matches_pattern 中对应的元素将是一个匹配对象（表示匹配成功），否则为 None（表示匹配失败）
        matches_pattern = [re.match(pattern, response, re.S) for response in responses]
        matches_pattern2 = [re.match(pattern2, response, re.S) for response in responses]

        format_rewards = []
        for match_pattern, match_pattern2 in zip(matches_pattern, matches_pattern2):
            # 如果匹配到正确的标签结构，奖励 0.5，否则奖励 0
            if match_pattern or match_pattern2:
                format_rewards.append(0.5)
            else:
                format_rewards.append(0.0)
        rewards += torch.tensor(format_rewards, device=args.device)

        # 进一步细化奖励：对于每个正确的标签结构，检查是否包含正确数量的 <think> 和 <answer> 标签，奖励 0.25 * 4 = 1.0 满分
        def mark_num(text):
            reward = 0
            # count 方法用于统计字符串中某个子字符串出现的次数
            # text.count("<think>") == 1 表示在生成的文本中恰好出现一次 <think> 标签
            # 如果满足条件则奖励增加 0.25 分，其他标签同理，最终如果四个标签都正确出现一次，则总奖励为 1.0 分
            if text.count("<think>") == 1: reward += 0.25
            if text.count("</think>") == 1: reward += 0.25
            if text.count("<answer>") == 1: reward += 0.25
            if text.count("</answer>") == 1: reward += 0.25
            return reward
        
        # 维度为 [batch_size * num_generations] 的列表，num_generations 是每个 prompt 生成的文本数量
        mark_rewards = [mark_num(response) for response in responses]  
        rewards += torch.tensor(mark_rewards, device=args.device)
        return rewards
    
    # 初始化奖励张量，维度为 [batch_size * num_generations]，初始值为 0
    rewards = torch.zeros(len(responses), device=args.device)
    # 如果启用推理模型奖励，则首先计算基于标签结构的奖励，然后在后续步骤中结合 reward 模型的评分进行综合奖励计算
    if args.reasoning == 1:
        rewards = reasoning_model_reward(rewards)

    with torch.no_grad():
        reward_model_scores = []
        batch_size = len(prompts)  # batch_size 是 prompt 的数量，每个 prompt 可能对应多个生成文本（num_generations）
        scale = 3.0  # 奖励缩放因子，控制 reward 模型评分的范围，避免过大或过小的奖励值对训练造成不稳定影响

        for i in range(batch_size):  # 遍历每个 prompt
            for j in range(args.num_generations):  # 遍历每个 prompt 对应的生成文本
                response_idx = i * args.num_generations + j  # 对应当前 prompt 和生成文本的索引位置
                response = responses[response_idx]
                prompt = prompts[i]

                # 解析对话格式
                pattern = r"<\|im_start\|>(system|user|assistant)\s+(.*?)<\|im_end\|>" # 正则表达式模式，用于匹配 prompt 
                # re.findall 返回一个列表，列表中的每个元素都是一个元组，包含匹配到的角色和内容
                matches = re.findall(pattern, prompt, re.DOTALL)  
                # 转化为标准的对话消息列表，每条消息包含角色和内容，供 reward 模型评分使用
                messages = [{"role": role, "content": content.strip()} for role, content in matches]
                # 将生成的文本作为 assistant 的回复添加到对话消息列表中，形成完整的对话上下文，供 reward 模型进行评分
                tmp_chat = messages + [{"role": "assistant", "content": response}]
                # 调用 reward 模型的 get_score 方法计算当前生成文本的奖励分数，输入是 reward_tokenizer 和构建的对话消息列表 tmp_chat
                score = reward_model.get_score(reward_tokenizer, tmp_chat) 
                score = max(min(score, scale), -scale)  # 对评分进行截断，确保奖励值在 [-scale, scale] 范围内

                if args.reasoning == 1:
                    # 从生成的文本中提取 <answer> 标签内的内容，并使用 reward 模型对该内容进行评分
                    answer_match = re.search(r'<answer>(.*?)</answer>', response, re.DOTALL)
                    if answer_match:
                        answer_content = answer_match.group(1).strip() # 提取 <answer> 标签内的内容并去除首尾空白字符
                        tmp_chat = messages + [{"role": "assistant", "content": answer_content}]
                        answer_score = reward_model.get_score(reward_tokenizer, tmp_chat)
                        answer_score = max(min(answer_score, scale), -scale)
                        # 综合考虑标签结构奖励和 reward 模型评分，计算最终奖励分数，权重分别为 0.4 和 0.6
                        score = score * 0.4 + answer_score * 0.6 

                reward_model_scores.append(score)

        # 维度为 [batch_size * num_generations] 
        reward_model_scores = torch.tensor(reward_model_scores, device=args.device)
        rewards += reward_model_scores

    return rewards

---

```python
pattern = r"<\|im_start\|>(system|user|assistant)\s+(.*?)<\|im_end\|>" # 正则表达式模式，用于匹配 prompt 
# re.findall 返回一个列表，列表中的每个元素都是一个元组，包含匹配到的角色和内容
matches = re.findall(pattern, prompt, re.DOTALL)  
# 转化为标准的对话消息列表，每条消息包含角色和内容，供 reward 模型评分使用
messages = [{"role": role, "content": content.strip()} for role, content in matches]
# 将生成的文本作为 assistant 的回复添加到对话消息列表中，形成完整的对话上下文，供 reward 模型进行评分
tmp_chat = messages + [{"role": "assistant", "content": response}]
```

举例解析以上代码：

原始输入可能如下所示（使用 ChatML 格式）：

```
<|im_start|>system
你是一个有帮助的助手。
<|im_end|>
<|im_start|>user
什么是光合作用？
<|im_end|>
```

正则表达式提取出每一段对话：

```python
matches = [
    ("system", "你是一个有帮助的助手。"),
    ("user", "什么是光合作用？")
]
```

然后转换为标准的消息格式：

```python
messages = [
    {"role": "system", "content": "你是一个有帮助的助手。"},
    {"role": "用户", "content": "什么是光合作用？"}
]
```

把模型的回答拼接到对话后面：
```python
tmp_chat = [
    {"role": "system", "content": "你是一个有帮助的助手。"},
    {"role": "user", "content": "什么是光合作用？"},
    {"role": "assistant", "content": "光合作用是植物利用..."} 
  ]
```

---

接下来，我们来看看 MiniMind 的训练函数（与`/minimind/train_grpo.py`中略有不同）

In [10]:
def grpo_train_epoch(epoch, loader, iters, ref_model, reward_model, reward_tokenizer, start_step=0, wandb=None):
    for step, batch in enumerate(loader, start=start_step + 1):
        prompts = batch['prompt']  # list[str], length B
        # input_ids: [B, P] → B个问题，每个P个token 
        # attention_mask: [B, P] → 标记哪些位置是真实token（1）哪些是padding（0）
        prompt_inputs = tokenizer(prompts, return_tensors="pt", padding=True, return_token_type_ids=False,
                                  padding_side="left", add_special_tokens=False).to(args.device)  
        if args.max_seq_len:
            # [;, -args.max_seq_len]表示保留输入序列的最后 max_seq_len 个 token
            prompt_inputs["input_ids"] = prompt_inputs["input_ids"][:, -args.max_seq_len:]
            prompt_inputs["attention_mask"] = prompt_inputs["attention_mask"][:, -args.max_seq_len:]

        with torch.no_grad():
            # DDP 模型需要使用 .module 访问 generate 方法
            model_for_gen = model.module if isinstance(model, DistributedDataParallel) else model
            outputs = model_for_gen.generate(  # [B*num_gen, P+R]，每条序列 = 问题部分(P个token) + 回答部分(R个token)
                **prompt_inputs,  # 展开 prompt_inputs 字典作为 generate 方法的输入参数，包括 input_ids 和 attention_mask
                max_new_tokens=args.max_gen_len,  # 生成文本最大长度
                do_sample=True,  # 使用采样而非贪婪解码，增加生成文本的多样性
                temperature=0.8,  # 控制生成文本的随机程度，值越大生成越随机，值越小生成越确定
                num_return_sequences=args.num_generations,  # 每个 prompt 生成的文本数量G
                pad_token_id=tokenizer.pad_token_id)  # 填充token ID，确保生成文本长度一致，便于后续处理

        completion_ids = outputs[:, prompt_inputs["input_ids"].size(1):]  # [B*num_gen, R]，只保留后面的 R 个 token（回答部分）
        
        def get_per_token_logps(mdl, input_ids, n_keep):
            input_ids = input_ids.detach().clone() if input_ids.is_inference() else input_ids
            # 通过模型前向传播获取 logits，logits 的维度为 [B*num_gen, P+R, vocab_size]
            # 由于我们只关心回答部分的 token 的 logp，因此限制模型只计算最后 n_keep + 1 个 token 的 logits
            # 然后通过切片 [:, :-1, :] 去掉最后一个 token 的 logits，得到回答部分的 logits
            logits = mdl(input_ids, logits_to_keep=n_keep + 1).logits[:, :-1, :]
            per_token_logps = []
            for logits_row, ids_row in zip(logits, input_ids[:, -n_keep:]):
                # ids_row 的维度为 [n_keep]，表示回答部分的 token ID 序列
                ids_row = ids_row.detach().clone() if ids_row.is_inference() else ids_row
                # logits_row.log_softmax(dim=-1) 将 logits 转为 log probabilities，维度为 [n_keep, vocab_size]
                # ids_row.unsqueeze(1) 将 ids_row 从 [n_keep] 变为 [n_keep, 1]，以便与 logits_row 的维度匹配
                # 通过 torch.gather 从 logits_row 中提取对应 ids_row 的 log probabilities
                # 最终得到 per_token_logps 的维度为 [n_keep]，表示回答部分每个 token 的 log probability
                per_token_logps.append(torch.gather(logits_row.log_softmax(dim=-1), 1, ids_row.unsqueeze(1)).squeeze(1))
            return torch.stack(per_token_logps)  # 堆叠成[B*num_gen, n_keep]

        with autocast_ctx:
            per_token_logps = get_per_token_logps(model, outputs, completion_ids.size(1))  # [B*num_gen, R]
            res = model(outputs) if lm_config.use_moe else None
            aux_loss = res.aux_loss if res is not None else torch.tensor(0.0, device=args.device)
        
        with torch.no_grad():
            ref_per_token_logps = get_per_token_logps(ref_model, outputs, completion_ids.size(1))  # [B*num_gen, R]

        # 把 token ID 解码回文字
        completions = tokenizer.batch_decode(completion_ids, skip_special_tokens=True)
        # 计算奖励值，维度为 [B*num_gen]，每个元素对应一个生成文本的奖励分数
        rewards = calculate_rewards(prompts, completions, reward_model, reward_tokenizer).to(args.device)  

        grouped_rewards = rewards.view(-1, args.num_generations)  # 重塑为[B, num_gen]
        mean_r = grouped_rewards.mean(dim=1).repeat_interleave(args.num_generations)  # 计算均值，重塑为[B*num_gen]
        std_r = grouped_rewards.std(dim=1).repeat_interleave(args.num_generations)  # 计算标准差，重塑为[B*num_gen]
        # 标准化奖励并裁剪，得到优势值，维度为 [B*num_gen]，组内归一化
        advantages = torch.clamp((rewards - mean_r) / (std_r + 1e-4), -10, 10)  
        # 进一步标准化优势值，增强训练稳定性，全局归一化
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)  

        is_eos = completion_ids == tokenizer.eos_token_id  # [B*num_gen, R]，为布尔张量，标记每个 token 是否为结束符
        # 计算每条生成文本的结束位置索引，默认值为 R（即假设没有结束符），如果存在结束符则取第一个结束符的位置
        eos_idx = torch.full((is_eos.size(0),), is_eos.size(1), dtype=torch.long, device=args.device)
        # is_eos.any(dim=1) 返回一个布尔张量，标记每条生成文本是否包含结束符
        # 通过 argmax 获取第一个结束符的位置索引，并更新 eos_idx 中对应位置的值，确保每条生成文本的结束位置正确标记
        # 例如，如果某条生成文本的 is_eos 行为 [False, False, True, False]，则 argmax 返回 2
        # 表示第一个结束符的位置索引为 2，eos_idx 中对应位置的值将被更新为 2
        eos_idx[is_eos.any(dim=1)] = is_eos.int().argmax(dim=1)[is_eos.any(dim=1)]
        # 生成一个回答的掩码，标记每个 token 是否在结束符位置之前，维度为 [B*num_gen, R]，值为 1 表示有效 token，值为 0 表示无效 token
        completion_mask = (torch.arange(is_eos.size(1), device=args.device).expand(is_eos.size(0), -1) <= eos_idx.unsqueeze(1)).int()  # [B*num_gen, R]

        # 参考模型和当前模型的 log 概率之差，维度为 [B*num_gen, R]
        kl_div = ref_per_token_logps - per_token_logps 
        # grpo 的KL惩罚项计算公式，维度为 [B*num_gen, R]，每个元素表示对应 token 的 KL 惩罚值
        per_token_kl = torch.exp(kl_div) - kl_div - 1  
        # 重要性采样的损失计算公式，维度为 [B*num_gen, R]，每个元素表示对应 token 的损失值
        # .detach() 的作用是"断开梯度"，让 per_token_logps.detach() 在反向传播时被当作常数
        per_token_loss = -(torch.exp(per_token_logps - per_token_logps.detach()) * advantages.unsqueeze(1) - args.beta * per_token_kl)  # [B*num_gen, R]
        policy_loss = ((per_token_loss * completion_mask).sum(dim=1) / completion_mask.sum(dim=1)).mean()
        loss = (policy_loss + aux_loss) / args.accumulation_steps  # scalar
        loss.backward()

        if (step + 1) % args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

        if step % args.log_interval == 0 or step == iters:
            policy_loss_val = loss.item() * args.accumulation_steps
            current_aux_loss = aux_loss.item()
            avg_reward_val = rewards.mean().item()
            avg_len_val = completion_mask.sum(dim=1).float().mean().item()
            current_lr = optimizer.param_groups[0]['lr']

            print(
                f'Epoch:[{epoch + 1}/{args.epochs}]({step}/{iters}), '
                f'Actor Loss: {policy_loss_val:.4f}, '
                f'Aux Loss: {current_aux_loss:.4f}, '
                f'Reward: {avg_reward_val:.4f}, '
                f'Avg Response Len: {avg_len_val:.2f}, '
                f'Learning Rate: {current_lr:.8f}'
            )

            if wandb:
                wandb.log({
                    "policy_loss": policy_loss_val,
                    "aux_loss": current_aux_loss,
                    "reward": avg_reward_val,
                    "avg_response_len": avg_len_val,
                    "advantages_mean": advantages.mean().item(),
                    "learning_rate": current_lr
                })

        # 到达指定保存步数时，保存模型（仅主进程）
        if args.save_interval > 0 and (step % args.save_interval == 0 or step == iters - 1):
            if not dist.is_initialized() or dist.get_rank() == 0:
                os.makedirs(args.save_dir, exist_ok=True)  # 确保保存目录存在
                model.eval()
                moe_suffix = '_moe' if lm_config.use_moe else ''
                ckp = f'{args.save_dir}/{args.save_weight}_{lm_config.dim}{moe_suffix}.pth'
                raw_model = model.module if isinstance(model, DistributedDataParallel) else model
                raw_model = getattr(raw_model, '_orig_mod', raw_model)
                state_dict = raw_model.state_dict()
                torch.save({k: v.half().cpu() for k, v in state_dict.items()}, ckp)
                print(f'模型已保存至：{ckp}')
                model.train()
                del state_dict

        del prompt_inputs, outputs, completion_ids, per_token_logps, ref_per_token_logps
        del completions, rewards, grouped_rewards, mean_r, std_r, advantages, completion_mask

准备完毕，我们尝试一轮长度 1 个 iter 的训练.

In [ ]:
iter_per_epoch = len(train_loader)
for epoch in range(args.epochs):
    grpo_train_epoch(epoch, train_loader, iter_per_epoch, ref_model, reward_model, reward_tokenizer)
print('grpo训练完成！')

由于grpo对GPU显存要求较高，因此不在这里展示演示结果

In [ ]:
del model